## Importing Libraries

In [1]:
import pandas as pd
import random as rd

## Loading actual relatable data

In [2]:
data = pd.read_csv('final.csv')

In [3]:
data.sample(5)

,CCMS ID,Active Power kW,Apparent Power kVA,Current R (A),Current Y (A),Current B (A),Voltage R (V),Voltage Y (V),Voltage B (V),Total PF,MD kVA,MD kW,R Phase kW,Y Phase kW,B Phase kW
438,SMR03-0219-0252,4.56,4.59,10.20,1.27,6.83,250.03,251.78,252.02,0.992 Ld,5.35,5.32,2.54,0.31,1.70
631,SMR03-0219-0252,4.69,4.71,10.88,1.36,7.71,236.61,236.57,235.90,0.994 Ld,5.35,5.32,2.56,0.31,1.81
167,SMR03-0219-0252,4.33,4.36,10.33,1.28,6.04,246.36,248.32,248.35,0.992 Ld,5.35,5.32,2.54,0.31,1.47
326,SMR03-0219-0252,4.60,4.63,10.91,1.30,6.97,240.46,242.46,242.44,0.992 Ld,5.35,5.32,2.62,0.31,1.67
390,SMR03-0219-0252,4.58,4.61,10.19,1.25,6.73,253.09,254.92,255.37,0.992 Ld,5.35,5.32,2.56,0.31,1.70


## Slicing to get current, voltage and total Power Factor which may help to predict fault location

In [4]:
data = data.iloc[:900, 3:10]

In [6]:
data.sample(5)

,Current R (A),Current Y (A),Current B (A),Voltage R (V),Voltage Y (V),Voltage B (V),Total PF
779,10.82,1.35,6.83,236.53,236.62,235.64,0.993 Ld
683,10.50,1.35,7.68,237.36,237.48,236.34,0.993 Ld
160,9.61,1.27,6.00,248.21,249.72,250.08,0.99 Ld
138,9.68,5.23,5.62,246.36,247.51,248.29,0.989 Ld
592,9.58,1.30,7.47,250.19,249.99,248.96,0.992 Ld


## Creating temporary fault location data as 1st feature

In [7]:
dist = [i*30 for i in range(1,31)]*30

In [8]:
dist.sort()

In [9]:
dist = dist*7

In [10]:
distance = pd.DataFrame(dist, columns=['distance'])

In [11]:
len(dist)

6300

## Creating a 2nd feature for wattage of different types of lamp

In [24]:
watt = [18, 24, 45, 70, 110, 140, 190]*1672

In [25]:
watt.sort()

In [26]:
watt = pd.DataFrame(watt[:11700], columns=['watt'])

In [27]:
len(watt)

11700

## Creating 3rd feature as total power factor

In [16]:
ind =0
temp=[]
for i in data.iloc[:, -1]:
    x =float(i[:5])
    temp.append(float("{0:.3f}".format(x)))
    ind  += 1
    

In [17]:
pf = temp*13

In [18]:
pf = pd.DataFrame(pf, columns=['PF'])

In [19]:
len(pf)

11700

## Creating other features of voltage and current 

In [23]:
Vr = []
Vy = []
Vb = []
Ir = []
Iy = []
Ib = []
for ind in range(7):
    i = watt[ind]/100
    v = watt[ind]/10
    
    for j in range(900):
        Ir.append(data.iloc[j, 0]+i)
        Iy.append(data.iloc[j, 1]+i)
        Ib.append(data.iloc[j, 2]+i)
        
        Vr.append(data.iloc[j, 3]+v)
        Vy.append(data.iloc[j, 4]+v)
        Vb.append(data.iloc[j, 5]+v)       
        

In [28]:
len(Vr)

6300

## Converting this into dataframe

In [29]:
Ir = pd.DataFrame(Ir, columns=['Ir'])
Iy = pd.DataFrame(Iy, columns=['Iy'])
Ib = pd.DataFrame(Ib, columns=['Ib'])

In [30]:
Vr = pd.DataFrame(Vr, columns=['Vr'])
Vy = pd.DataFrame(Vy, columns=['Vy'])
Vb = pd.DataFrame(Vb, columns=['Vb'])

## Final concatination

In [31]:
ccms_data = pd.concat([watt,Ir,Iy,Ib,Vr,Vy,Vb,pf,distance], axis = 1)

In [32]:
ccms_data.head()

,watt,Ir,Iy,Ib,Vr,Vy,Vb,PF,distance
0,18,10.84,5.65,6.03,235.28,236.37,237.20,0.992,30.0
1,18,10.83,5.66,6.01,235.36,236.14,237.25,0.992,30.0
2,18,10.92,5.70,6.07,233.54,234.52,235.52,0.992,30.0
3,18,10.91,5.71,6.08,233.81,235.02,235.94,0.992,30.0
4,18,10.97,5.73,6.10,234.97,236.07,236.96,0.989,30.0


## Downloading as a csv file

In [ ]:
ccms_data.to_csv('ccms_data.csv')